## Import Modules & Load Data

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython
!pip install -q -U keras-tuner
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

img_train.shape, img_test.shape

((60000, 28, 28), (10000, 28, 28))

## Model Definition

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model



## Instantiate Tuner & Hyper Tuning

In [5]:
tuner = kt.Hyperband(model_builder, 
                     objective = 'val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [6]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [7]:
tuner.search(img_train, label_train, epochs=10, validation_data=(img_test, label_test),
             callbacks=[ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4799 - accuracy: 0.8289 - val_loss: 0.4254 - val_accuracy: 0.8445
Epoch 2/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3613 - accuracy: 0.8684 - val_loss: 0.4186 - val_accuracy: 0.8493
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3253 - accuracy: 0.8815 - val_loss: 0.3723 - val_accuracy: 0.8666
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2999 - accuracy: 0.8883 - val_loss: 0.3706 - val_accuracy: 0.8631
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2834 - accuracy: 0.8947 - val_loss: 0.3404 - val_accuracy: 0.8810
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2682 - accuracy: 0.9004 - val_loss: 0.3463 - val_accuracy: 0.8799
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2558 - accuracy: 0.9043 - val_loss: 0.3406 - val_accurac

In [ ]:
model2 = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model2.compile(optimizer = 'adam',
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy']
)
model2.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))